In [ ]:
import re
import numpy as np
import pandas as pd

def assign_state_and_value(s, val):
    m = re.match(r'(.+) [–-] (.+)', s.state)
    if not m:
        print(s.state)
    s['state'] = m[1].strip()
    s[val] = m[2]
    return s

def replace_with_length(df, cols):
    for col in cols:
        df = df.assign(**{col: df[col].map(lambda x: float(len(x)) if not pd.isna(x) else x)})
    return df

cases = pd.read_csv('raw/sheet-1.csv')
vaxed = pd.read_csv('raw/sheet-2.csv')

cases = cases.apply(lambda s: assign_state_and_value(s, 'covid_cases'), 1)
cases['covid_cases'] = cases.covid_cases.map(lambda s: int(s))

vaxed = vaxed.apply(lambda s: assign_state_and_value(s, 'vaxed'), 1)
vaxed['vaxed'] = vaxed.vaxed.map(lambda s: float(re.sub('%', '', s)))

check_cols = ['political_will', 'technical_capacity', 'social_media']

df = cases.merge(vaxed, how='outer', on=['state', 'political_will', 'technical_capacity', 'social_media', 'zone'])
df = df.fillna('⦸')
df = df.replace('FCT', 'Federal Capital Territory')

In [ ]:
from environs import Env
from adopt.malaria import load_basics, get_db_conf
from adopt.campaign_queries import get_campaigns_for_user, create_campaign_confs, create_campaign_for_user

env = Env()
env.read_env('.env', override=True)
db_conf = get_db_conf(env)

USER = "malaria2@vlab.digital"
CAMPAIGN = "nigeria-healthworkers-preliminary"
AD_ACCOUNT = '2661361017437088'

CAMPAIGNID = next(c['id'] for c in get_campaigns_for_user(USER, db_conf) 
                  if c['name'] == CAMPAIGN)

userinfo, config, db_conf, state, m, confs = load_basics(CAMPAIGNID, env)

In [ ]:
def get_api(env, token: str) -> FacebookAdsApi:
    session = FacebookSession(env("FACEBOOK_APP_ID"), env("FACEBOOK_APP_SECRET"), token)
    api = FacebookAdsApi(session)
    return api

In [ ]:
from facebook_business.adobjects.targetingsearch import TargetingSearch

state_locs = [(s, TargetingSearch.search({'type': 'adgeolocation', 'q': s, 'location_types': ['region'], 'country_code': 'NG'}, api=state.api))
             for s in df.state]


In [186]:
state_targets = [(name, l[0]) for name, l in state_locs]

In [224]:
from adopt.facebook.state import CampaignState, get_api

template_state = CampaignState(userinfo.token, 
                               get_api(env, userinfo.token), 
                               AD_ACCOUNT, 
                               'scratch')

In [228]:
hcp_spec = template_state.adsets[0]['targeting']['flexible_spec']

In [ ]:
TargetingSearch.search({'type': 'adgeolocation', 'q': 'Federal Capital Territory', 'location_types': ['region'], 'country_code': 'NG'}, api=state.api)

In [239]:

def estimate_reach(name, targeting_region, gender=0, hcp=False):
    spec = {
        'geo_locations':{
            'regions': [targeting_region.export_all_data()]
        },
        'genders': [gender]
    }

    if hcp == True:
        spec['flexible_spec'] = hcp_spec


    params =  { 'targeting_spec': spec }
    est = state.account.get_reach_estimate(params=params)

    users = est[0]['users']
    return name, users


reach = {
    'fb_reach': [estimate_reach(*s, 0, False) for s in state_targets],
    'hcp_reach': [estimate_reach(*s, 0, True) for s in state_targets],
    'hcp_reach_female': [estimate_reach(*s, 2, True) for s in state_targets]
}

In [361]:
reach_df = pd.DataFrame({k: dict(v) for k, v in reach.items()}).reset_index().rename(columns={'index': 'state'})

dd = df.merge(reach_df).sort_values('hcp_reach', ascending=False).reset_index(drop=True)

In [362]:
pop = pd.read_csv('../../power/nga_admpop_adm1_2020.csv')
pop = pop[pop.ADM1_NAME.notna()]
pop['state'] = pop.ADM1_NAME.str.title()
pop = pop[['state', 'T_TL']]

In [363]:
dd.loc[dd.state == 'Nassarawa', 'state'] = 'Nasarawa'
dd.loc[dd.state == 'Akwa-Ibom', 'state'] = 'Akwa Ibom'

In [ ]:
dd = dd.merge(pop, how='left')
dd['population'] = ((dd.T_TL / 100_000).round(0) * 100_000).astype(int)
dd = dd.drop(columns=['T_TL'])

In [365]:
pov = pd.read_excel('raw/nga-subnational-results-mpi-2020.xlsx', skiprows=9, usecols=range(6,9), header=None, names=['state', 'country_average', 'poverty_index']).drop(columns=['country_average'])
pov.loc[pov.state == 'FCT', 'state'] = 'Federal Capital Territory'
pov['poverty_index'] = pov.poverty_index.round(2)

In [366]:
# pd.read_csv('../../power/nigeriahealthfacilities.csv')

In [367]:
dd = dd.merge(pov, how='left')

In [375]:
dd = dd.sort_values(['hcp_reach', 'fb_reach'], ascending=False)

dd['fb_penetration'] = (dd.fb_reach / dd.population).round(2)
dd['vaxed'] = (dd.vaxed / 100).round(2)

In [ ]:
dd

In [377]:
dd.to_csv('nigeria-hcp-state-data.csv', index=False)

In [386]:
dat = dd[dd.vaxed <= 1.0]

In [ ]:
dat

In [391]:
w = dat.population / dat.population.sum()
p = dat.vaxed @ w

dat.vaxed.var() / (p*(1-p))

0.054680085970142636

In [ ]:
# df = df.replace('⦸', np.nan)
# df = replace_with_length(df, check_cols)

# df

